In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""


import pandas as pd
import tensorflow as tf
# import theano
import keras 

import numpy as np
import matplotlib.pyplot as plt
from time import time

Using TensorFlow backend.


In [2]:
!nvidia-smi

Wed Oct 25 11:28:19 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.82                 Driver Version: 375.82                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 750 Ti  Off  | 0000:02:00.0     Off |                  N/A |
| N/A   37C    P8    N/A /  N/A |   1843MiB /  2002MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
def show(id, w=6):
    from IPython.display import Image
    return Image('./pics/class1/{}.jpg'.format(id), width=w*100)

In [3]:
df = pd.read_csv('../Artificial_Neural_Networks/Churn_Modelling.csv')
df.shape

(10000, 14)

In [4]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [5]:
df.iloc[:,3:-1].head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [6]:
df.iloc[:,3:13].head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [7]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [8]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

### data preprocessing

In [9]:
X = df.iloc[:,3:-1].values
print X.shape
print type(X)
X[:3]

(10000, 10)
<type 'numpy.ndarray'>


array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57]], dtype=object)

In [10]:
y = df.iloc[:,-1].values
print y.shape
print type(y)
y

(10000,)
<type 'numpy.ndarray'>


array([1, 0, 1, ..., 1, 1, 0])

### Encoding categorical data

In [11]:
X[:,1:3]

array([['France', 'Female'],
       ['Spain', 'Female'],
       ['France', 'Female'],
       ..., 
       ['France', 'Female'],
       ['Germany', 'Male'],
       ['France', 'Female']], dtype=object)

In [12]:
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
print X[:3]
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
X[:3]

[[619 0 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 2 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 0 'Female' 42 8 159660.8 3 1 0 113931.57]]


array([[619, 0, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 2, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 0, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57]], dtype=object)

In [13]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ..., 
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [14]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print X.shape

(10000, 12)


In [15]:
onehotencoder.fit_transform(X)

<10000x13 sparse matrix of type '<type 'numpy.float64'>'
	with 91124 stored elements in COOrdinate format>

In [16]:
# %precision % .2f

https://stackoverflow.com/questions/2891790/how-to-pretty-printing-a-numpy-array-without-scientific-notation-and-with-given

In [17]:
np.set_printoptions(precision=1, suppress=True)

In [18]:
onehotencoder.fit_transform(X).toarray()[0]

array([      1. ,       0. ,       1. ,       0. ,     619. ,       0. ,
            42. ,       2. ,       0. ,       1. ,       1. ,       1. ,
        101348.9])

### remove one dummy variable to avoid dummy variable trap

In [19]:
X[0]

array([      1. ,       0. ,       0. ,     619. ,       0. ,      42. ,
             2. ,       0. ,       1. ,       1. ,       1. ,  101348.9])

In [20]:
print X.shape
X = X[:, 1:]
print X.shape
X[0]

(10000, 12)
(10000, 11)


array([      0. ,       0. ,     619. ,       0. ,      42. ,       2. ,
             0. ,       1. ,       1. ,       1. ,  101348.9])

### train test split & feature scaling 

In [21]:
S = lambda *x: [i.shape for i in x]

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print S(X_train,X_test, y_train, y_test)

[(8000, 11), (2000, 11), (8000,), (2000,)]


In [23]:
X_train[0]

array([      0. ,       1. ,     667. ,       0. ,      34. ,       5. ,
             0. ,       2. ,       1. ,       0. ,  163830.6])

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print S(X_train,X_test, y_train, y_test)
X_train[0]

[(8000, 11), (2000, 11), (8000,), (2000,)]


array([-0.6,  1.7,  0.2, -1.1, -0.5,  0. , -1.2,  0.8,  0.6, -1. ,  1.1])

## build ANN

In [25]:
show(13, w=8)

In [26]:
from keras.models import Sequential
from keras.layers import Dense

In [27]:
classifier = Sequential()
# classifier?

In [28]:
# first hidden layer
# classifier.add(Dense(units = 6, input_shape=(11,),  kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units = 6, input_dim=11,  kernel_initializer='uniform', activation='relu'))

In [29]:
# second hidden layer
classifier.add(Dense(units = 6,  kernel_initializer='uniform', activation='relu'))

In [30]:
# ouput layer
classifier.add(Dense(units = 1,  kernel_initializer='uniform', activation='sigmoid'))

In [31]:
show(19, w=8)

In [32]:
# compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [33]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [34]:
!ps aux --sort=-%mem | awk 'NR<=10{print $0}'

USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root     22243  0.9  3.5 1010728 581552 ?      Sl   11:01   0:12 gunicorn: worker [app:app]                                                                  
soyoung  21057  8.5  1.8 2241032 301008 ?      Ssl  11:22   0:03 /home/saya/anaconda2/bin/python -m ipykernel_launcher -f /home/soyoung/.local/share/jupyter/runtime/kernel-983046a1-c51a-4757-aba4-85f60e627114.json
soyoung   1439  0.2  0.9 627312 148124 ?       Sl   Sep30  95:29 /home/saya/anaconda2/bin/python /home/saya/anaconda2/bin/jupyter-notebook
iri       3251  0.3  0.7 2325252 114664 ?      Sl   Oct17  38:59 /usr/lib/firefox/firefox
soyoung  26672  0.0  0.3 32553672 55064 ?      S    Oct23   0:00 /home/saya/anaconda2/bin/python -m ipykernel_launcher -f /home/soyoung/.local/share/jupyter/runtime/kernel-073405de-4a55-4dac-b813-986a4167411c.json
soyoung  26673  0.0  0.3 32553676 55064 ?      S    Oct23   0:00 /home/saya/anaconda2/bin/python -m ipykernel_la

In [37]:
tf.all_variables()

Instructions for updating:
Please use tf.global_variables instead.


[<tf.Variable 'dense_1/kernel:0' shape=(11, 6) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(6, 6) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'dense_3/kernel:0' shape=(6, 1) dtype=float32_ref>,
 <tf.Variable 'dense_3/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'Adam/iterations:0' shape=() dtype=int64_ref>,
 <tf.Variable 'Adam/lr:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Adam/beta_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Adam/beta_2:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Adam/decay:0' shape=() dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable:0' shape=(11, 6) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_1:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_2:0' shape=(6, 6) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_3:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'training/

In [41]:
tf.

In [35]:
# fitting ANN with training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 0s - loss: 0.4787 - acc: 0.7959     
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 0.4263 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - ETA: 0s - loss: 0.4198 - acc: 0.805 - 0s - loss: 0.4207 - acc: 0.8061     
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 0.4179 - acc: 0.8247     
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 0.4161 - acc: 0.8271     
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 0.4144 - acc: 0.8296     
Epoch 7/100
8000/8000 [==============================] - 0s - loss: 0.4128 - acc: 0.8307     
Epoch 8/100
8000/8000 [==============================] - 0s - loss: 0.4115 - acc: 0.8319     
Epoch 9/100
8000/8000 [==============================] - 0s - loss: 0.4110 - acc: 0.8329     
Epoch 10/100
8000/8000 [==============================] - 0s - loss: 0.4092 - acc: 0.8339     
Epoch 11/100
8000/800

KeyboardInterrupt: 

In [39]:
y_pred = classifier.predict(X_test)
print type(y_pred)
y_pred.shape

<type 'numpy.ndarray'>


(2000, 1)

In [43]:
y_pred

array([[ 0.2],
       [ 0.3],
       [ 0.2],
       ..., 
       [ 0.2],
       [ 0.2],
       [ 0.1]], dtype=float32)

In [44]:
y_pred[0][0]

0.21550092

In [47]:
y_pred = (y_pred> 0.5)
y_pred

array([[False],
       [False],
       [False],
       ..., 
       [False],
       [False],
       [False]], dtype=bool)

In [50]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
type(cm)

numpy.ndarray

In [49]:
cm

array([[1512,   83],
       [ 194,  211]])

In [51]:
accuracy_score(y_test, y_pred)

0.86150000000000004

In [53]:
(1512+211)/2000.0

0.8615

In [55]:
classifier.save?

### Predicting a single new observation

In [59]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]], dtype=bool)

## cross validation